In [129]:
import pandas as pd
import numpy as np
from datetime import datetime
# pd.options.display.max_rows = 100 
# pd.options.display.max_columns = 50
pd.set_option('display.max_rows', 500)

In [130]:
# url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.csv"
# disasterDataDf = pd.read_csv(url, index_col='state')

data = pd.read_csv('data.csv', index_col='incidentType')

In [131]:
data = data.drop(['id','lastRefresh','hash','declarationRequestNumber','placeCode','fipsCountyCode','fipsStateCode','hmProgramDeclared','paProgramDeclared','iaProgramDeclared','ihProgramDeclared','fyDeclared','disasterNumber','disasterNumber','declarationType','incidentBeginDate','incidentEndDate','designatedArea'],axis = 1).fillna('')
data.head(20)

,femaDeclarationString,state,declarationDate,declarationTitle,disasterCloseoutDate
incidentType,,,,,
Tornado,DR-7-MA,MA,1953-06-11T00:00:00.000Z,TORNADO,1956-06-01T00:00:00.000Z
Flood,DR-9-TX,TX,1953-06-19T00:00:00.000Z,FLOOD,1958-01-01T00:00:00.000Z
Flood,DR-8-IA,IA,1953-06-11T00:00:00.000Z,FLOOD,1955-11-01T00:00:00.000Z
Fire,DR-11-NH,NH,1953-07-02T00:00:00.000Z,FOREST FIRE,1956-02-01T00:00:00.000Z
Tornado,DR-6-MI,MI,1953-06-09T00:00:00.000Z,TORNADO,1956-03-30T00:00:00.000Z
Flood,DR-5-MT,MT,1953-06-06T00:00:00.000Z,FLOODS,1955-12-01T00:00:00.000Z
Flood,DR-3-LA,LA,1953-05-29T00:00:00.000Z,FLOOD,1960-02-01T00:00:00.000Z
Tornado,DR-4-MI,MI,1953-06-02T00:00:00.000Z,TORNADO,1956-02-01T00:00:00.000Z
Tornado,DR-1-GA,GA,1953-05-02T00:00:00.000Z,TORNADO,1954-06-01T00:00:00.000Z


In [132]:
def get_closeout_duration(df):
#     df = pd.concat([declaration, closeout], axis=1)
#     df.replace('NaN', '', regex=True)
    df = df.applymap(lambda x: datetime.strptime(x[:x.index('T')], '%Y-%m-%d') if x != '' else datetime.today().date())
    return (df['disasterCloseoutDate'] - df['declarationDate']).apply(lambda x: x.days)

In [133]:
data[['declarationDate', 'disasterCloseoutDate']] = data[['declarationDate', 'disasterCloseoutDate']].applymap(lambda date: datetime.strptime(date[:date.index('T')], '%Y-%m-%d') if date != '' else datetime.today().date())
data['closeoutDuration'] = data['disasterCloseoutDate'] - data['declarationDate']
# data['closeoutDuration'] = (data['disasterCloseoutDate'] - data['declarationDate']).apply(lambda x: x.days)
# data['closeoutDuration'] = getCloseoutDuration(data[['declarationDate', 'disasterCloseoutDate']])

In [134]:
data

,femaDeclarationString,state,declarationDate,declarationTitle,disasterCloseoutDate,closeoutDuration
incidentType,,,,,,
Tornado,DR-7-MA,MA,1953-06-11,TORNADO,1956-06-01,1086 days
Flood,DR-9-TX,TX,1953-06-19,FLOOD,1958-01-01,1657 days
Flood,DR-8-IA,IA,1953-06-11,FLOOD,1955-11-01,873 days
Fire,DR-11-NH,NH,1953-07-02,FOREST FIRE,1956-02-01,944 days
Tornado,DR-6-MI,MI,1953-06-09,TORNADO,1956-03-30,1025 days
...,...,...,...,...,...,...
Severe Storm(s),EM-3561-FL,FL,2021-07-04,TROPICAL STORM ELSA,2021-07-12,8 days
Severe Storm(s),EM-3561-FL,FL,2021-07-04,TROPICAL STORM ELSA,2021-07-12,8 days
Severe Storm(s),EM-3561-FL,FL,2021-07-04,TROPICAL STORM ELSA,2021-07-12,8 days


In [147]:
data.sort_values('closeoutDuration', ascending=False).groupby(['femaDeclarationString']).head(1).groupby(['incidentType']).head(5)

,femaDeclarationString,state,declarationDate,declarationTitle,disasterCloseoutDate,closeoutDuration
incidentType,,,,,,
Toxic Substances,EM-3066-NY,NY,1978-08-07,"CHEMICAL WASTE, LOVE CANAL",2011-09-30,12107 days
Flood,DR-792-NY,NY,1987-05-15,FLOODING,2011-09-30,8904 days
Flood,DR-483-PR,PR,1975-09-19,TROPICAL STORM ELIOISE,1999-07-14,8699 days
Earthquake,DR-1008-CA,CA,1994-01-17,NORTHRIDGE EARTHQUAKE,2017-09-29,8656 days
Fire,FM-2196-HI,HI,1998-03-16,LEILANI ESTATES SUBDIVISION FIRE (HAWAII CO),2021-07-12,8519 days
Severe Storm(s),DR-1236-WI,WI,1998-07-24,"SEVERE STORMS,STRAIGH LINE WINDS, TORNADOES,RA...",2021-07-12,8389 days
Toxic Substances,EM-3080-NY,NY,1980-05-21,"CHEMICAL WASTE, LOVE CANAL",2003-01-15,8274 days
Hurricane,DR-1067-VI,VI,1995-09-16,HURRICANE MARILYN,2017-10-26,8076 days
Hurricane,DR-841-VI,VI,1989-09-20,HURRICANE HUGO,2011-08-22,8006 days


In [ ]:
data.groupby(['incidentType','femaDeclarationString']).a